In [ ]:
%reset

In [ ]:
import pdb; pdb.set_trace()
#%pylab inline
%reload_ext autoreload
%autoreload 2
%load_ext memory_profiler
%load_ext line_profiler
from qoreliquid import *
from qore_qstk import *
from pylab import rcParams
import pandas as p
rcParams['figure.figsize'] = 20, 13

out = getEfficientFrontierCharts(dt.datetime(2014,9,11), calculateHowMany=None, printHowMany=None, days=365*1, annotate=True)
#calculateEfficientFrontier(5, dt.datetime.now())

In [ ]:
%pylab inline
%reload_ext autoreload
%autoreload 2
%load_ext memory_profiler
%load_ext line_profiler
from qoreliquid import *
from qore_qstk import *
from pylab import rcParams
import pandas as p
rcParams['figure.figsize'] = 20, 13

In [ ]:
#print searchQuandl('SP500', returndataset=True)

#print getDataFromQuandl('YAHOO/SP500_5010TR', dataset='')
#print getDataFromQuandl('YAHOO/INDEX_GSPC', dataset='')

zxc = getDataFromQuandl('YAHOO/INDEX_GSPC', dataset='')
#zxc = getDataFromQuandl('YAHOO/SP500_5010TR', dataset='')
zxc['Close'].plot()

In [ ]:
symb1 = 'GOOG'
%mprun getDatasetSymbol(symb1).plot(); show();

In [ ]:
%lprun -f generateQstkSymbolsTxt generateQstkSymbolsTxt()

In [ ]:
out = getEfficientFrontierCharts(dt.datetime(2014,9,11), calculateHowMany=None, printHowMany=None, days=365*1, annotate=True)
#calculateEfficientFrontier(5, dt.datetime.now())

In [ ]:
#trollPlots()
#searchQuandl('renewable')

In [ ]:
#dt_end=dt.datetime(2012,1,1)
dt_end=dt.datetime.now()
portfolioBacktester(dt_end=dt_end)

In [ ]:
qu = ['EIA/SEDS_REPRB_WA_A']
#qu = 'EIA/SEDS_REPRB_WA_A'
#getDataFromQuandl(qu)
plot1 = getDataFromQuandl(qu, plot=True)


In [ ]:
# Portfolio Backtester
# Tips for accessing historical data via DataAccess + a quick and dirty portfolio back test
# source: http://wiki.quantsoftware.org/index.php?title=QSTK_Tutorial_3

import QSTK.qstkutil.qsdateutil as du
import QSTK.qstkutil.tsutil as tsu
import QSTK.qstkutil.DataAccess as da
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

csv = """symbol, allocation
SPY,0.3
GABBABOOM,0.2
GLD,0.3
7ABBA, 0.2
"""
fname = 'tutorial3portfolio.csv'
fp = open(fname, 'w')
fp.write(csv)
fp.close()

#Reading in the portfolio description

#NumPy provides a nice utility, loadtxt() for reading in CSV formatted data files. Here's the code for reading in the portfolio:
na_portfolio = np.loadtxt(fname, dtype='S5,f4',
                        delimiter=',', comments="#", skiprows=1)
os.remove(fname)
print p.DataFrame(na_portfolio)

#The second line (dtype=) defines the format for each column. I think the other arguments are self explanatory. Now let's take a look at what we get back from this read:
#[('SPY', 0.30000001192092896) ('GABBA', 0.20000000298023224),('GLD', 0.30000001192092896) ('7ABBA', 0.20000000298023224)]
#Later on it will be helpful if our data is sorted by symbol name, so we'll do that next:
na_portfolio = sorted(na_portfolio, key=lambda x: x[0])
print p.DataFrame(na_portfolio)
#Which prints out:
#[('7ABBA', 0.20000000298023224), ('GABBA', 0.20000000298023224), ('GLD', 0.30000001192092896), ('SPY', 0.30000001192092896)]
#Now we build two lists, one that contains the symbols and one that contains the allocations:
ls_port_syms = []
lf_port_alloc = []
for port in na_portfolio:
    ls_port_syms.append(port[0])
    lf_port_alloc.append(port[1])
#Checking for spurious symbols and removing them
#
#Now we're going to benefit from the horsepower of our DataAccess class and Python's set operations. First step is to see which symbols are available, then intersect that list with the symbols in our portfolio:
c_dataobj = da.DataAccess('Yahoo')
ls_all_syms = c_dataobj.get_all_symbols()
ls_bad_syms = list(set(ls_port_syms) - set(ls_all_syms))
#The second line above returns a list of all symbols available to us in the "Yahoo" data store. On the third line above we convert the list of all symbols, and the list of symbols in our portfolio into sets, then remove the symbols not present in the ls_all_syms but present in the ls_port_syms. These are the bad symbols.

if len(ls_bad_syms) != 0:
        print "Portfolio contains bad symbols : ", ls_bad_syms
#The above code results in the following print out:
#Portfolio contains bad symbols : ['7ABBA', 'GABBA']
#Now we'll remove those bad symbols from our portfolio:
for s_sym in ls_bad_syms:
    i_index = ls_port_syms.index(s_sym)
    ls_port_syms.pop(i_index)
    lf_port_alloc.pop(i_index)
#Configuring times and reading the data
#
#The list portsyms now contains the proper list of valid symbols, so we can ask DataAccess to return them for us with out blowing up. First we must set up the time boundaries as below:
dt_end = dt.datetime(2011, 1, 1)
dt_start = dt_end - dt.timedelta(days=1095)  # Three years
dt_timeofday = dt.timedelta(hours=16)

ldt_timestamps = du.getNYSEdays(dt_start, dt_end, dt_timeofday)

ls_keys = ['open', 'high', 'low', 'close', 'volume', 'actual_close']

ldf_data = c_dataobj.get_data(ldt_timestamps, ls_port_syms, ls_keys)
d_data = dict(zip(ls_keys, ldf_data))
#The code above reads in the data for the symbols in our portfolio between the dates of Jan 1, 2011, back to 1095 days before that (3 years).
#Now, a quick and dirty back test
#
#Note: this example computes portfolio returns assuming daily rebalancing. For coursera homework 1, you should not assume daily rebalancing.
#The first step is to prep the data. We make a copy of our closing prices in to "rets", fill the data forward, then convert it into daily returns:
df_rets = d_data['close'].copy()
df_rets = df_rets.fillna(method='ffill')
df_rets = df_rets.fillna(method='bfill')

na_rets = df_rets.values
tsu.returnize0(na_rets)
#Note that we extracted an ndarray from "close" (a pandas DataFrame), so we're now no longer benefitting from DataFrame features. You should consult other locations on this site for details on fill forward and converting into daily returns. For our combined portfolio we'll assume the combined return for each day is a sum of the returns for each equity weighted by the allocation. We can quickly compute the daily returns and the cumulative returns as follows:
na_portrets = np.sum(na_rets * lf_port_alloc, axis=1)
#print 'na_portrets'; print na_portrets
na_port_total = np.cumprod(na_portrets + 1)
#print 'na_port_total'; print na_port_total
#In a similar manner we can compute the returns of the individual components as follows:
na_component_total = np.cumprod(na_rets + 1, axis=0)
#That's it for the "back test." porttot contains the total returns for our combined portfolio.
#Plotting the results

#Our combined portfolio (and component equities).
plt.clf()
fig = plt.figure()
fig.add_subplot(111)
plt.plot(ldt_timestamps, na_component_total, alpha=0.4)
plt.plot(ldt_timestamps, na_port_total)
ls_names = ls_port_syms
ls_names.append('Portfolio')
plt.title('Portfolio Allocations incl. assets');
plt.legend(ls_names,2)
plt.ylabel('Cumulative Returns')
plt.xlabel('Date'); plt.show();
fig.autofmt_xdate(rotation=45);
plt.plot(ldt_timestamps, na_port_total); 
plt.title('Portfolio Allocations ex. assets');
plt.legend('Portfolio',2); plt.show();
#plt.savefig('tutorial3.pdf', format='pdf')


In [ ]:
# The Allocation DataFrame
# source: http://wiki.quantsoftware.org/index.php?title=QSTK_Tutorial_4

#An allocation DataFrame is created using the pandas data structure DataFrame. 
#This structure consists of a series of python datetime objects as indexes, 
#and stock symbols for column headings. The last column in the DataFrame is the symbol _CASH, 
#which represents what percentage of a portfolio is held in cash. 
#Each row has the distribution of the portfolio recorded via values in each column 
#that represent how much of the portfolio is in that stock at that date and time. 
#Normally these values are normalized so that they add to one.

#Creating an Allocation

#In order to setup an allocation table, we must first create the DataFrame. 
#A pandas DataFrame object consists of indices, column headings, and data. 
#Datetime objects are used for the indices. We can create a list of timestamps over the time 
#period we are interested in looking in a similar fashion to how we produce timestamps 
#for the DataAccess utility. We create a list of timestamps like so:
dt_start = dt.datetime(2004, 1, 1)
dt_end = dt.datetime(2009, 12, 31)

# We need closing prices so the timestamp should be hours=16.
dt_timeofday = dt.timedelta(hours=16)

# Get a list of trading days between the start and the end.
ldt_timestamps = du.getNYSEdays(dt_start, dt_end, dt_timeofday)

# Creating an object of the dataaccess class with Yahoo as the source.
c_dataobj = da.DataAccess('Yahoo')
#These timestamps will help us look at the dates between 2004 through the end of 2009.
#Symbols

#The allocation DataFrame describes the distribution of a portfolio across several stocks. 
#It also details how much of the portfolio should be in cash at a point in time. 
#In this particular example, we wish to look at the first 20 stocks of the S&P500. 
#The way we set up column headers to reflect this stock symbol information is like so:
ls_symbols = c_dataobj.get_symbols_from_list('sp5002012')
ls_symbols = ls_symbols[:20]
ls_symbols.append('_CASH')


#Allocations

#Now that we have the symbols and time period for our allocation, we must determine how much of the portfolio
#will be in each equity at each date. Normally this will be the most involved part in creating an allocation. 
#In this particular example we will be creating a new random distribution for our portfolio 
#at the start of each month. We create random values for the first row in our allocation table like so:
na_vals = np.random.randint(0, 1000, len(ls_symbols))
# Normalize the row - Typecasting as everything is int.
na_vals = na_vals / float(sum(na_vals))
# Reshape to a 2D matrix to append into dataframe.
na_vals = na_vals.reshape(1, -1)
#We then make a one row DataFrame for the first date using the constructor like so:
df_alloc = pd.DataFrame(na_vals, index=[ldt_timestamps[0]], columns=ls_symbols)
#Since we only desire to reallocate our portfolio once a month, we only add a new row to the DataFrame 
#for each new month. We must then repeat the process of creating and normalizing random values for the allocation. 
#Then we create another DataFrame row and append it to what we have already. 
#Thus we build up an allocation table across all of the months in our list of timestamps.
dt_last_date = ldt_timestamps[0]
# Looping through all dates and creating monthly allocations
for dt_date in ldt_timestamps[1:]:
#    if dt_last_date.month != dt_date.month:
    # Create allocation
    na_vals = np.random.randint(0, 1000, len(ls_symbols))
    na_vals = na_vals / float(sum(na_vals))
    na_vals = na_vals.reshape(1, -1)
    # Append to the dataframe
    df_new_row = pd.DataFrame(na_vals, index=[dt_date], columns=ls_symbols)
    df_alloc = df_alloc.append(df_new_row)
        
    dt_last_date = dt_date
    
#print n.sum(df_alloc, axis=1)
print df_alloc.ix[0,:]
#df_alloc.plot(style='.'); show()
#print output

#Using an Allocation

#QSTK uses allocation tables for a variety of tasks, most notably in backtesting strategies. 
#In order to use an allocation table, you can either pass it to the function you wish to use it with, 
#or store it in a pickle file for later usage. To do the latter, you must import the cPickle package and 
#dump the object like so:
#output=open("allocations.pkl","wb")
#cPickle.dump(alloc,output)

In [ ]:
df_alloc0 = p.DataFrame(df_alloc.ix[0,:].get_values(), index=df_alloc.ix[0,:].index)
#print df_alloc0
#print 1 - df_alloc.ix[:,0].get_values()
#df_alloc0[1] = df_alloc.ix[:,0].get_values() - n.sum(df_alloc0, axis=0)[0]
#print df_alloc.ix[0,:].to_csv(fname)
#print na_portfolio